In [337]:
# Suzan Iloglu, May 21,2020
# Import packages
import csv
import gurobipy as gp
from itertools import product
import geopandas as gpd
import pandas as pd
import numpy as np
import math
import time
import requests
import io
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
pd.options.display.max_columns =200
from IPython.display import Image


# MAPPING THE NEW POLITICS OF CARE: COMMUNITY HEALTH WORKERS
The project presents multiple options for how individual workers in such a Community Health Corps might be distributed within each state. It shows that what you choose to prioritize greatly impacts where care would be sent. We can define communities in greatest need in many ways: we can think about our current crisis and send people to where the COVID19 pandemic rages most fiercely; we can think of long term measures of social and economic inequality embedded in metrics like the Centers for Disease Control and Prevention’s Social Vulnerability Index; we can focus on the places with too many people dying too young and use the County Health Rankings Years-of-Potential-Life-Lost measure; we can think of joblessness and how the pandemic has thrown many into unemployment and target our resources in this way. 

The followings are our options to choose to define vulnerability:


- SOCIAL VULNERABILITY INDEX
- MEDICAID 
- UNEMPLOYMENT
- YEARS OF POTENTIAL LIFE LOST
- TOTAL COVID CASES
- COVID CASES BY POPULATION
- COVID DEATHS BY POPULATION

We will start with Social Vulnerability Index (SVI) from CDC website.

### I. Importing SVI data which includes the variables for calculating county SVI for each state
The CDC uses both a USA-wide and a state by state SVI scores. For our project given that funding is likely going to be managed at a state level, using a state by state SVI scores makes the most sense and will be most sensitive to regional socioeconomic differences. Even though the CDC SVI scores are calculated using percentile rankings, the data sets include raw data estimates for each variables. The following table shows the variablaes used in the method of calculating SVI scores. 




      American Community Survey (ACS), 2014-2018 (5-year) data for the following estimates:
<img src="Data/img/SVI_comp.png" width="500">


Note: Full documentation for 2018 data is available <a href="https://svi.cdc.gov/data-and-tools-download.html">here</a> 
This part of the code shows preliminary mapping of <a href = "https://svi.cdc.gov/">the CDC's Social Vulnerability Index</a>.

Later in the notebook, we will provide the formula to create the SVI value we use in our project. First, we import the data for the US mainland and Puerto Rico.

In [338]:
## import svi data downloaded from CDC website as cited above

## 48 state SVI scores by county
svi_counties_mainland = gpd.read_file("Data/SVI2018_US_COUNTY/SVI2018_US_county.shp")

## Puerto Rico SVI scores by county
svi_counties_puerto_rico = gpd.read_file("Data/PuertoRico_COUNTY/SVI2018_PuertoRico_county.shp")

## Merge 48 states and Puerto Rico SVI 
svi_counties = pd.concat([svi_counties_mainland,svi_counties_puerto_rico ], sort = False)


In [339]:
## Replacing -999 values with 0 for calculations
svi_county = svi_counties.fillna(0)
svi_county  = svi_county.replace(-999, 0)
svi_county['FIPS'] = svi_county['FIPS'].astype(int)

In [340]:
## Create the list for State
State = svi_county.STATE.unique().tolist()

In [341]:
# Create a seperate dictionary for the variables to calculate SVI

# Persons below poverty estimate, 2014-2018 ACS
E_POV = dict(zip(svi_county.FIPS, svi_county.E_POV))

# Civilian (age 16+) unemployed estimate, 2014-2018 ACS
E_UNEMP = dict(zip(svi_county.FIPS, svi_county.E_UNEMP))

# Per capita income estimate, 2014-2018 ACS
E_PCI = dict(zip(svi_county.FIPS, svi_county.E_PCI))

# Persons (age 25+) with no high school diploma estimate, 2014-2018 ACS
E_NOHSDP = dict(zip(svi_county.FIPS, svi_county.E_NOHSDP))

# Persons aged 65 and older estimate
E_AGE65 = dict(zip(svi_county.FIPS, svi_county.E_AGE65))

# Persons aged 17 and younger estimate
E_AGE17 = dict(zip(svi_county.FIPS, svi_county.E_AGE17))

# Population with a disability estimate
E_DISABL = dict(zip(svi_county.FIPS, svi_county.E_DISABL))

# Single parent households with children under 18 estimate
E_SNGPNT = dict(zip(svi_county.FIPS, svi_county.E_SNGPNT))

# Minority (all persons except white, nonHispanic) estimate, 2014-2018 ACS
E_MINRTY = dict(zip(svi_county.FIPS, svi_county.E_MINRTY))

# Persons (age 5+) who speak English "less than well" estimate, 2014-2018 ACS
E_LIMENG = dict(zip(svi_county.FIPS, svi_county.E_LIMENG))

# Housing in structures with 10 or more units estimate, 2014-2018 ACS
E_MUNIT = dict(zip(svi_county.FIPS, svi_county.E_MUNIT))

# Mobile homes estimate MOE, 2014-2018 ACS
E_MOBILE = dict(zip(svi_county.FIPS, svi_county.E_MOBILE))

# At household level (occupied housing units), more people than rooms estimate, 2014-2018 ACS
E_CROWD = dict(zip(svi_county.FIPS, svi_county.E_CROWD))

# Households with no vehicle available estimate, 2014-2018 ACS
E_NOVEH = dict(zip(svi_county.FIPS, svi_county.E_NOVEH))

# Persons in institutionalized group quarters estimate, 2014-2018 ACS
E_GROUPQ = dict(zip(svi_county.FIPS, svi_county.E_GROUPQ))

# Percentage of persons below poverty estimate
E_POV = dict(zip(svi_county.FIPS, svi_county.E_POV))

# Medicaid 
Medicaid is a means-tested health insurance program for low-income children, pregnant women, adults, seniors, and people with disabilities. Medicaid is jointly funded by federal and state governments and managed by states within federal standards and a wide range of state options. <a href="https://data.medicaid.gov/Enrollment/State-Medicaid-and-CHIP-Applications-Eligibility-D/n5ce-jxme"> Data Source for Medicaid Enrollment </a> 

In [342]:

import sodapy
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.medicaid.gov", None)


# Returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("83yt-67it", limit=4000)


# Read the medicaid demand data
#df_mm = pd.read_csv("Data/2020_06_Preliminary_applications__eligibility_determinations__and_enrollment_data.csv")

# Convert to pandas DataFrame
df_mm = pd.DataFrame.from_records(results)
df_mm.head(5)
df_mm.columns


Index(['applications_for_financial_assistance_submitted_to_the_state_based_marketplace',
       'applications_for_financial_assistance_submitted_to_the_state_based_marketplace_footnotes',
       'final_report', 'geocoded_column',
       'individuals_determined_eligible_for_chip_at_application',
       'individuals_determined_eligible_for_chip_at_application_footnotes',
       'individuals_determined_eligible_for_medicaid_at_application',
       'individuals_determined_eligible_for_medicaid_at_application_footnotes',
       'latitude', 'longitude', 'medicaid_and_chip_child_enrollment',
       'medicaid_and_chip_child_enrollment_footnotes',
       'new_applications_submitted_to_medicaid_and_chip_agencies',
       'new_applications_submitted_to_medicaid_and_chip_agencies_footnotes',
       'preliminary_updated', 'report_date', 'state_abbreviation',
       'state_expanded_medicaid', 'state_name',
       'total_applications_for_financial_assistance_submitted_at_state_level',
       'total_a

In [343]:
df_mm['State Name'] = df_mm['state_name'].str.upper() 

In [344]:
Medicaid_state = dict(zip(df_mm['State Name'], df_mm['total_medicaid_and_chip_enrollment']))
Medicaid_state['PUERTO RICO'] = 1622194
print (Medicaid_state)

{'ALABAMA': '957116', 'ALASKA': '231145', 'ARIZONA': '1839932', 'ARKANSAS': '830467', 'CALIFORNIA': '11847711', 'COLORADO': '1337805', 'CONNECTICUT': '874974', 'DELAWARE': '239009', 'DISTRICT OF COLUMBIA': '250860', 'FLORIDA': '3892552', 'GEORGIA': '1928703', 'HAWAII': '351337', 'IDAHO': '340742', 'ILLINOIS': '2987496', 'INDIANA': '1602976', 'IOWA': '699741', 'KANSAS': '401103', 'KENTUCKY': '1416013', 'LOUISIANA': '1585024', 'MAINE': '232455', 'MARYLAND': '1372695', 'MASSACHUSETTS': '1616404', 'MICHIGAN': '2439425', 'MINNESOTA': '1085778', 'MISSISSIPPI': '632427', 'MISSOURI': '923641', 'MONTANA': '257006', 'NEBRASKA': '254159', 'NEVADA': '685073', 'NEW HAMPSHIRE': '193436', 'NEW JERSEY': '1759653', 'NEW MEXICO': '772102', 'NEW YORK': '6263164', 'NORTH CAROLINA': '1851558', 'NORTH DAKOTA': '96757', 'OHIO': '2788134', 'OKLAHOMA': '797220', 'OREGON': '1053931', 'PENNSYLVANIA': '3069309', 'RHODE ISLAND': '305208', 'SOUTH CAROLINA': '1048276', 'SOUTH DAKOTA': '114059', 'TENNESSEE': '1489536

In [345]:
df_mmm = pd.read_csv("Data/ACSST5Y2019.S2704_data_with_overlays_2020-12-11T225641.csv", header=[1])
df_mmm.head(1)
#df_mmm.dtypes

,id,Geographic Area Name,Estimate!!Total!!Civilian noninstitutionalized population,Margin of Error!!Total!!Civilian noninstitutionalized population,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!Under 19,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!Under 19,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!19 to 64 years,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!19 to 64 years,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!65 years and over,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!65 years and over,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!Under 19,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!Under 19,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!19 to 64 years,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!19 to 64 years,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!65 years and over,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!65 years and over,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!Under 19,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!Under 19,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!19 to 64 years,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!19 to 64 years,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!65 years and over,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!65 years and over,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Below 138 percent of the poverty threshold,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Below 138 percent of the poverty threshold,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!At or above 138 percent of the poverty threshold,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!At or above 138 percent of the poverty threshold,"Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Worked full-time, year-round (19-64 years)","Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Worked full-time, year-round (19-64 years)",Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Under 6,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Under 6,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!6 to 18 years,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!6 to 18 years,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALON

In [346]:
df_mmm['FIPS'] = df_mmm.id.astype(str).str[9:]
df_mmm.head(5)
df_mmm.FIPS.astype(int)
df_mmm['FIPS'] = pd.to_numeric(df_mmm['FIPS'])

In [347]:
# 2018 ACI data
# ACI_total  = dict(zip(df_mmm['FIPS'], df_mmm["Estimate!!Public Coverage!!Medicaid/means-tested public coverage alone or in combination"]))

#2019 ACI data
ACI_total  = dict(zip(df_mmm['FIPS'], df_mmm["Estimate!!Public Coverage!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination"]))




In [348]:

#df_m = pd.read_csv("Data/Medicaid_Demand.csv")

# Unemployment 
The unemployment rate is calculated by the U.S. Bureau of Labor Statistics as the percentage of the civilian labor force who are without jobs and have actively sought work within the past four weeks. <a href="https://www.bls.gov/lau/laufaq.htm#Q01"> Data Source for Unemployment  </a> 

In [349]:
from io import StringIO
import datetime 
from datetime import date
from dateutil.relativedelta import relativedelta

back = date.today() + relativedelta(months= -3)
three_months_ago = back.strftime('%b-%y') 


url = 'https://www.bls.gov/web/metro/laucntycur14.txt'
s = requests.get(url).text



df_unemp = pd.read_csv(StringIO(s), sep='|',  skiprows=7, skipfooter=6, engine='python', names = ['LAUS Area Code', 'FIPS State', 'FIPS County', 'Area Title', 'Period', 'Civilian Labor Force','Employed','Unemployed_Level','Unemployed_Rate'])

df_unemp['Period'] = df_unemp['Period'].astype(str)

df_unemp['FIPS'] = df_unemp['LAUS Area Code'].str[3:8]

df_unemp = df_unemp[df_unemp['Period'].str.contains(str(three_months_ago))]

df_unemp.head(5)
#df_unemp.dtypes

,LAUS Area Code,FIPS State,FIPS County,Area Title,Period,Civilian Labor Force,Employed,Unemployed_Level,Unemployed_Rate,FIPS
38627,CN0100100000000,1,1,"Autauga County, AL",Nov-20,"25,334","24,482",852,3.4,01001
38628,CN0100300000000,1,3,"Baldwin County, AL",Nov-20,"92,160","88,745","3,415",3.7,01003
38629,CN0100500000000,1,5,"Barbour County, AL",Nov-20,"9,733","9,190",543,5.6,01005
38630,CN0100700000000,1,7,"Bibb County, AL",Nov-20,"8,627","8,251",376,4.4,01007
38631,CN0100900000000,1,9,"Blount County, AL",Nov-20,"24,849","24,210",639,2.6,01009


In [350]:
#df_unemp[[ 'FIPS', 'Area Title', 'Period','Unemployed_Level','Unemployed_Rate']].to_csv('Data/County_employment.csv', index=False)


In [351]:


df_unemp.replace({'-', 0})

df_unemp['Unemployed_Level'] = df_unemp['Unemployed_Level'].str.replace(',', '')

df_unemp['Unemployed_Rate'] = df_unemp['Unemployed_Rate'].astype(str)

df_unemp['Unemployed_Level'] = df_unemp['Unemployed_Level'].str.strip()

df_unemp['Unemployed_Rate'] = df_unemp['Unemployed_Rate'].str.strip()

df_unemp['FIPS'] = pd.to_numeric(df_unemp['FIPS'])

df_unemp['Unemployed_Level'] = pd.to_numeric(df_unemp['Unemployed_Level'])

df_unemp['Unemployed_Rate'] = pd.to_numeric(df_unemp['Unemployed_Rate'])




# Fill NA with 0
#df_unemp = df_unemp.fillna(0)
df_unemp.tail(5)
#df_unemp.dtypes


,LAUS Area Code,FIPS State,FIPS County,Area Title,Period,Civilian Labor Force,Employed,Unemployed_Level,Unemployed_Rate,FIPS
41841,CN7214500000000,72,145,"Vega Baja Municipio, PR",Nov-20,"12,447","10,911",1536,12.3,72145
41842,CN7214700000000,72,147,"Vieques Municipio, PR",Nov-20,"2,445","2,166",279,11.4,72147
41843,CN7214900000000,72,149,"Villalba Municipio, PR",Nov-20,"6,613","5,859",754,11.4,72149
41844,CN7215100000000,72,151,"Yabucoa Municipio, PR",Nov-20,"7,907","7,027",880,11.1,72151
41845,CN7215300000000,72,153,"Yauco Municipio, PR",Nov-20,"9,202","8,089",1113,12.1,72153


In [352]:
 
## Read the Unemployment data using cvs data 
#df_unemp = pd.read_csv("Data/Unemployment.csv")

# Fill NA with 0
#df_unemp = df_unemp.fillna(0)

#df_unemp.head(5)

# Years of Potential Life Lost (YPLL)

Years of Potential Life Lost (YPLL) measures the rate of premature deaths by region. YPLL is calculated as the sum of the estimated number of years that individuals would have lived if they had not died before the age of 75 per 100,000 people. <a href="https://www.countyhealthrankings.org/sites/default/files/media/document/2020%20County%20Health%20Rankings%20Data%20-%20v2.xlsx"> Data Source for YPLL.  </a> More information about YPLL can be dounf in this <a href="https://www.countyhealthrankings.org/explore-health-rankings/measures-data-sources/county-health-rankings-model/health-outcomes/length-of-life/premature-death-ypll"> link. </a> 


In [353]:
# Read the YPLL data
df_y = pd.read_csv("Data/YPLL.csv")

# Fill NA with the mean of the data
df_y = df_y.fillna(df_y.mean())


# Population

In [354]:
# Read the Population data
df_pop = pd.read_csv("Data/County_pop_2019.csv")

# Fill NA with 0
df_pop = df_pop.fillna(0)


In [355]:
# Create a dictionary for the county and population
population_county = df_pop.set_index('FIPS')['pop'].to_dict()

# Create a dictionary for the county and YPLL
YPLL = dict(zip(df_y.FIPS, df_y.YPLL))

# Create a dictionary for the county and Unemployment
Unemployment = dict(zip(df_unemp.FIPS, df_unemp.Unemployed_Level))


# Create a dictionary for the county and Community Health Workers (CHW) demand
# Note that we assume a CHW can serve 55 Medicaid patient so the demand for CHW will be

#Medicaid_demand = dict(zip(df_m.FIPS, df_m.Med_Demand))


In [356]:
#for m in Medicaid_demand:
#    print (m, Medicaid_demand[m])

# COVID-19 Cases & COVID-19 Cases per Capita

What are COVID-19 Cases and COVID-19 Cases per Capita?

COVID-19 cases is an absolute metric of the total number of COVID-19 cases in a county over the last fourteen days.  COVID-19 cases per 100,000 is a relative metric calculated by dividing the number of COVID-19 cases by the estimated county population and multiplying by 100,000.  Cases include both confirmed cases, based on viral testing, and probable cases, based on specific criteria for symptoms and epidemiological exposure. We use NY Times Covid data. 


In [357]:
#### Data with the most recent date in NY Times dataset:

today = time.strftime('%Y-%m-%d')
covid_data_update_date = today#'2020-07-21'#today #or enter a specific date such as '2020-07-06'


In [358]:
## 14 day period defined
data_date_dt = pd.to_datetime(covid_data_update_date,infer_datetime_format = True)

N = 14

date_N_days_ago = data_date_dt - timedelta(days = N)

date_N1_days_ago = data_date_dt - timedelta(days = N+1)

In [359]:

# URL for mainland US data
url = "http://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"
s = requests.get(url).content
covid = pd.read_csv(io.StringIO(s.decode('utf-8')))

In [360]:
covid.tail(100)

,date,county,state,fips,cases,deaths
1043666,2021-02-18,Webster,West Virginia,54101.0,285,2.0
1043667,2021-02-18,Wetzel,West Virginia,54103.0,1057,19.0
1043668,2021-02-18,Wirt,West Virginia,54105.0,341,2.0
1043669,2021-02-18,Wood,West Virginia,54107.0,6888,128.0
1043670,2021-02-18,Wyoming,West Virginia,54109.0,1712,33.0
1043671,2021-02-18,Adams,Wisconsin,55001.0,1721,15.0
1043672,2021-02-18,Ashland,Wisconsin,55003.0,1234,16.0
1043673,2021-02-18,Barron,Wisconsin,55005.0,5414,75.0
1043674,2021-02-18,Bayfield,Wisconsin,55007.0,1130,18.0
1043675,2021-02-18,Brown,Wisconsin,55009.0,32295,227.0


Note: Since NY data is seperately available, we first read the NY data for all 5 different borough then combine with the rest of US data.

In [361]:
# URL for NY
url = "https://raw.githubusercontent.com/nychealth/coronavirus-data/master/trends/data-by-day.csv"
#"https://raw.githubusercontent.com/nychealth/coronavirus-data/master/data-by-day.csv"

ny = requests.get(url).content
covid_ny = pd.read_csv(io.StringIO(ny.decode('utf-8')))


covid_ny.tail(5)

,date_of_interest,CASE_COUNT,PROBABLE_CASE_COUNT,HOSPITALIZED_COUNT,DEATH_COUNT,PROBABLE_DEATH_COUNT,CASE_COUNT_7DAY_AVG,ALL_CASE_COUNT_7DAY_AVG,HOSP_COUNT_7DAY_AVG,DEATH_COUNT_7DAY_AVG,ALL_DEATH_COUNT_7DAY_AVG,BX_CASE_COUNT,BX_PROBABLE_CASE_COUNT,BX_HOSPITALIZED_COUNT,BX_DEATH_COUNT,BX_PROBABLE_DEATH_COUNT,BX_CASE_COUNT_7DAY_AVG,BX_ALL_CASE_COUNT_7DAY_AVG,BX_HOSPITALIZED_COUNT_7DAY_AVG,BX_DEATH_COUNT_7DAY_AVG,BX_ALL_DEATH_COUNT_7DAY_AVG,BK_CASE_COUNT,BK_PROBABLE_CASE_COUNT,BK_HOSPITALIZED_COUNT,BK_DEATH_COUNT,BK_PROBABLE_DEATH_COUNT,BK_CASE_COUNT_7DAY_AVG,BK_ALL_CASE_COUNT_7DAY_AVG,BK_HOSPITALIZED_COUNT_7DAY_AVG,BK_DEATH_COUNT_7DAY_AVG,BK_ALL_DEATH_COUNT_7DAY_AVG,MN_CASE_COUNT,MN_PROBABLE_CASE_COUNT,MN_HOSPITALIZED_COUNT,MN_DEATH_COUNT,MN_PROBABLE_DEATH_COUNT,MN_CASE_COUNT_7DAY_AVG,MN_ALL_CASE_COUNT_7DAY_AVG,MN_HOSPITALIZED_COUNT_7DAY_AVG,MN_DEATH_COUNT_7DAY_AVG,MN_ALL_DEATH_COUNT_7DAY_AVG,QN_CASE_COUNT,QN_PROBABLE_CASE_COUNT,QN_HOSPITALIZED_COUNT,QN_DEATH_COUNT,QN_PROBABLE_DEATH_COUNT,QN_CASE_COUNT_7DAY_AVG,QN_ALL_CASE_COUNT_7DAY_AVG,QN_HOSPITALIZED_COUNT_7DAY_AVG,QN_DEATH_COUNT_7DAY_AVG,QN_ALL_DEATH_COUNT_7DAY_AVG,SI_CASE_COUNT,SI_PROBABLE_CASE_COUNT,SI_HOSPITALIZED_COUNT,SI_DEATH_COUNT,SI_PROBABLE_DEATH_COUNT,SI_CASE_COUNT_7DAY_AVG,SI_ALL_CASE_COUNT_7DAY_AVG,SI_HOSPITALIZED_COUNT_7DAY_AVG,SI_DEATH_COUNT_7DAY_AVG,SI_ALL_DEATH_COUNT_7DAY_AVG,INCOMPLETE
349,02/12/2021,3213,908,321,64,13,3224,4242,334,74,81,700,205,78,11,0,720,933,88,16,16,914,257,93,19,3,939,1238,97,22,23,480,148,42,6,0,460,630,48,11,11,899,245,85,23,0,897,1168,82,20,21,219,53,23,5,0,208,272,19,5,5,11000
350,02/13/2021,2096,861,236,62,17,3092,4098,317,72,80,396,171,75,14,0,675,885,86,16,17,623,250,61,19,1,904,1198,92,21,22,259,156,37,7,2,442,611,44,10,11,687,230,55,17,1,869,1138,78,20,20,131,54,8,5,0,201,265,17,4,4,11000
351,02/14/2021,1755,845,217,59,15,3113,4128,305,70,79,351,164,65,13,0,681,891,84,15,16,542,249,66,18,1,902,1199,88,21,22,234,117,30,11,0,447,613,42,10,11,517,252,43,15,0,879,1156,73,19,19,111,63,13,2,0,203,269,17,4,4,11000
352,02/15/2021,2517,689,168,52,23,2858,3793,277,66,78,444,120,55,9,2,610,796,77,14,14,773,208,35,17,2,826,1099,78,20,21,326,82,21,11,0,407,559,38,10,10,765,230,46,13,0,817,1081,69,18,19,209,49,11,2,0,197,257,16,4,4,11000
353,02/16/2021,2815,881,25,48,24,2760,3656,228,63,78,598,179,14,13,0,578,754,65,14,15,842,268,7,20,0,801,1064,65,20,21,455,148,3,3,0,404,550,30,8,9,773,227,1,11,1,792,1043,55,17,17,145,59,0,1,0,185,245,13,4,4,11000


In [362]:
Kings = covid_ny[['date_of_interest', 'BK_CASE_COUNT', 'BK_DEATH_COUNT']]
Kings.rename(columns = {'BK_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'BK_DEATH_COUNT': 'deaths'} , inplace=True)
Kings['county'] = 'Kings'
Kings['state'] = 'New York'
Kings['fips'] = 36047.0
#Kings.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

In [363]:
Bronx = covid_ny[['date_of_interest', 'BX_CASE_COUNT', 'BX_DEATH_COUNT']]
Bronx.rename(columns = {'BX_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'BX_DEATH_COUNT': 'deaths'} , inplace=True)
Bronx['state'] = 'New York'
Bronx['county'] = 'Bronx'
Bronx['fips'] = 36005.0
#Bronx.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [364]:
Manhattan = covid_ny[['date_of_interest', 'MN_CASE_COUNT', 'MN_DEATH_COUNT']]
Manhattan.rename(columns = {'MN_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'MN_DEATH_COUNT': 'deaths'} , inplace=True)
Manhattan['state'] = 'New York'
Manhattan['county'] = 'Manhattan'
Manhattan['fips'] = 36061.0
#Manhattan.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [365]:
Queens = covid_ny[['date_of_interest', 'QN_CASE_COUNT', 'QN_DEATH_COUNT']]
Queens.rename(columns = {'QN_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'QN_DEATH_COUNT': 'deaths'} , inplace=True)
Queens['state'] = 'New York'
Queens['county'] = 'Queens'
Queens['fips'] = 36081.0
#Queens.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [366]:
Richmond = covid_ny[['date_of_interest', 'SI_CASE_COUNT', 'SI_DEATH_COUNT']]
Richmond.rename(columns = {'SI_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'SI_DEATH_COUNT': 'deaths'} , inplace=True)
Richmond['state'] = 'New York'
Richmond['county'] = 'Richmond'
Richmond['fips'] = 36085.0

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [367]:
covid.head(5)


,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0


In [368]:
Bronx.tail(5)

,date,cases,deaths,state,county,fips
349,02/12/2021,700,11,New York,Bronx,36005.0
350,02/13/2021,396,14,New York,Bronx,36005.0
351,02/14/2021,351,13,New York,Bronx,36005.0
352,02/15/2021,444,9,New York,Bronx,36005.0
353,02/16/2021,598,13,New York,Bronx,36005.0


In [369]:
#Kings['deaths'] = Kings.groupby(by=['fips'])['deaths_d'].sum()
#Bronx['deaths'] = Bronx.groupby(by=['fips'])['deaths_d'].sum()
#Manhattan['deaths'] = Manhattan.groupby(by=['fips'])['deaths_d'].sum()
#Queens['deaths'] = Queens.groupby(by=['fips'])['deaths_d'].sum()
#Richmond['deaths'] = Richmond.groupby(by=['fips'])['deaths_d'].sum()

In [370]:
Kings.head(5)

,date,cases,deaths,county,state,fips
0,02/29/2020,0,0,Kings,New York,36047.0
1,03/01/2020,0,0,Kings,New York,36047.0
2,03/02/2020,0,0,Kings,New York,36047.0
3,03/03/2020,0,0,Kings,New York,36047.0
4,03/04/2020,1,0,Kings,New York,36047.0


In [371]:
#Kings = Kings.drop(['deaths_d'], axis=1)
#Bronx = Bronx.drop(['deaths_d'], axis=1)
#Manhattan = Manhattan.drop(['deaths_d'], axis=1)
#Queens = Queens.drop(['deaths_d'], axis=1)
#Richmond = Richmond.drop(['deaths_d'], axis=1)

In [372]:
Kings.tail(5)

,date,cases,deaths,county,state,fips
349,02/12/2021,914,19,Kings,New York,36047.0
350,02/13/2021,623,19,Kings,New York,36047.0
351,02/14/2021,542,18,Kings,New York,36047.0
352,02/15/2021,773,17,Kings,New York,36047.0
353,02/16/2021,842,20,Kings,New York,36047.0


In [373]:
covid['dt'] = pd.to_datetime(covid['date'], infer_datetime_format=True)
Kings['dt'] = pd.to_datetime(Kings['date'], infer_datetime_format=True)
Bronx['dt'] = pd.to_datetime(Bronx['date'], infer_datetime_format=True)
Manhattan['dt'] = pd.to_datetime(Manhattan['date'], infer_datetime_format=True)
Queens['dt'] = pd.to_datetime(Queens['date'], infer_datetime_format=True)
Richmond['dt'] = pd.to_datetime(Richmond['date'], infer_datetime_format=True)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [374]:

Kings_c = Kings[(Kings['dt']>date_N1_days_ago) & (Kings['dt']<= data_date_dt)].copy()
Queens_c = Queens[(Queens['dt']>date_N1_days_ago) & (Queens['dt']<= data_date_dt)].copy()
Bronx_c = Bronx[(Bronx['dt']>date_N1_days_ago) & (Bronx['dt']<= data_date_dt)].copy()
Manhattan_c = Manhattan[(Manhattan['dt']>date_N1_days_ago) & (Manhattan['dt']<= data_date_dt)].copy()
Richmond_c = Richmond[(Richmond['dt']>date_N1_days_ago) & (Richmond['dt']<= data_date_dt)].copy()

In [375]:
#Kings['cases'] = Kings.loc[(Kings['dt']<= data_date_dt)].groupby(['dt'])['d_cases'].sum()
#Kings['cases'] = Kings.apply(lambda x: x[(Kings['dt']<= data_date_dt)]['d_cases'].sum())

In [376]:
Kings.head(50)

,date,cases,deaths,county,state,fips,dt
0,02/29/2020,0,0,Kings,New York,36047.0,2020-02-29
1,03/01/2020,0,0,Kings,New York,36047.0,2020-03-01
2,03/02/2020,0,0,Kings,New York,36047.0,2020-03-02
3,03/03/2020,0,0,Kings,New York,36047.0,2020-03-03
4,03/04/2020,1,0,Kings,New York,36047.0,2020-03-04
5,03/05/2020,3,0,Kings,New York,36047.0,2020-03-05
6,03/06/2020,1,0,Kings,New York,36047.0,2020-03-06
7,03/07/2020,2,0,Kings,New York,36047.0,2020-03-07
8,03/08/2020,5,0,Kings,New York,36047.0,2020-03-08
9,03/09/2020,16,0,Kings,New York,36047.0,2020-03-09


In [377]:
K_c = Kings_c.groupby(['fips']).sum().reset_index()
Q_c = Queens_c.groupby(['fips']).sum().reset_index()
B_c = Bronx_c.groupby(['fips']).sum().reset_index()
M_c = Manhattan_c.groupby(['fips']).sum().reset_index()
R_c = Richmond_c.groupby(['fips']).sum().reset_index()
K_c.head(19)

,fips,cases,deaths
0,36047.0,10568,260


In [378]:
covid_death = covid[(covid['dt'] >= data_date_dt- timedelta(days = 1))]
Kings_death = Kings.groupby(by=['fips'])['deaths'].sum().reset_index()
Bronx_death = Bronx.groupby(by=['fips'])['deaths'].sum().reset_index()
Manhattan_death = Manhattan.groupby(by=['fips'])['deaths'].sum().reset_index()
Queens_death = Queens.groupby(by=['fips'])['deaths'].sum().reset_index()
Richmond_death = Richmond.groupby(by=['fips'])['deaths'].sum().reset_index()

In [379]:
# We merge the NY data with the rest of the US data
covid = pd.concat([covid, Kings, Bronx, Manhattan, Queens, Richmond], sort = False)


In [380]:
Queens.tail(14)

,date,cases,deaths,state,county,fips,dt
340,02/03/2021,1373,20,New York,Queens,36081.0,2021-02-03
341,02/04/2021,1382,25,New York,Queens,36081.0,2021-02-04
342,02/05/2021,1156,20,New York,Queens,36081.0,2021-02-05
343,02/06/2021,881,23,New York,Queens,36081.0,2021-02-06
344,02/07/2021,446,20,New York,Queens,36081.0,2021-02-07
345,02/08/2021,1202,16,New York,Queens,36081.0,2021-02-08
346,02/09/2021,949,20,New York,Queens,36081.0,2021-02-09
347,02/10/2021,963,21,New York,Queens,36081.0,2021-02-10
348,02/11/2021,939,20,New York,Queens,36081.0,2021-02-11
349,02/12/2021,899,23,New York,Queens,36081.0,2021-02-12


In [381]:

covid.tail(50)

,date,county,state,fips,cases,deaths,dt
304,12/29/2020,Richmond,New York,36085.0,584,9.0,2020-12-29
305,12/30/2020,Richmond,New York,36085.0,462,4.0,2020-12-30
306,12/31/2020,Richmond,New York,36085.0,386,3.0,2020-12-31
307,01/01/2021,Richmond,New York,36085.0,122,3.0,2021-01-01
308,01/02/2021,Richmond,New York,36085.0,508,8.0,2021-01-02
309,01/03/2021,Richmond,New York,36085.0,396,5.0,2021-01-03
310,01/04/2021,Richmond,New York,36085.0,654,8.0,2021-01-04
311,01/05/2021,Richmond,New York,36085.0,515,5.0,2021-01-05
312,01/06/2021,Richmond,New York,36085.0,523,6.0,2021-01-06
313,01/07/2021,Richmond,New York,36085.0,452,7.0,2021-01-07


In [382]:
print (data_date_dt)

2021-02-19 00:00:00


In [383]:

covid_death = covid_death[['fips', 'deaths']]
covid_death.head(2)

,fips,deaths
1040521,1001.0,84.0
1040522,1003.0,254.0


In [384]:
Kings_death.head(2)

,fips,deaths
0,36047.0,7120


In [385]:
# We merge the NY data with the rest of the US data
covid_death = pd.concat([covid_death, Kings_death, Bronx_death, Manhattan_death, Queens_death, Richmond_death], sort = False)

covid_death = covid_death.rename(columns={"deaths": "total_deaths"})


# Create a dictionary for the cumulative COVID deaths in each county
County_covid_death = dict(zip(covid_death.fips, covid_death.total_deaths))

covid_death.tail(250)

,fips,total_deaths
1043521,51690.0,44.0
1043522,51115.0,7.0
1043523,51117.0,46.0
1043524,51119.0,16.0
1043525,51121.0,71.0
1043526,51125.0,7.0
1043527,51127.0,7.0
1043528,51700.0,116.0
1043529,51710.0,153.0
1043530,51131.0,32.0


In [386]:
covid_death.shape

(3250, 2)

In [387]:
print (County_covid_death)

{1001.0: 84.0, 1003.0: 254.0, 1005.0: 49.0, 1007.0: 57.0, 1009.0: 121.0, 1011.0: 33.0, 1013.0: 64.0, 1015.0: 265.0, 1017.0: 98.0, 1019.0: 37.0, 1021.0: 92.0, 1023.0: 23.0, 1025.0: 48.0, 1027.0: 53.0, 1029.0: 35.0, 1031.0: 91.0, 1033.0: 117.0, 1035.0: 23.0, 1037.0: 22.0, 1039.0: 98.0, 1041.0: 51.0, 1043.0: 171.0, 1045.0: 105.0, 1047.0: 134.0, 1049.0: 170.0, 1051.0: 171.0, 1053.0: 60.0, 1055.0: 300.0, 1057.0: 53.0, 1059.0: 75.0, 1061.0: 67.0, 1063.0: 32.0, 1065.0: 62.0, 1067.0: 40.0, 1069.0: 251.0, 1071.0: 95.0, 1073.0: 1319.0, 1075.0: 32.0, 1077.0: 205.0, 1079.0: 85.0, 1081.0: 141.0, 1083.0: 127.0, 1085.0: 44.0, 1087.0: 42.0, 1089.0: 422.0, 1091.0: 52.0, 1093.0: 92.0, 1095.0: 195.0, 1097.0: 697.0, 1099.0: 37.0, 1101.0: 476.0, 1103.0: 239.0, 1105.0: 27.0, 1107.0: 52.0, 1109.0: 70.0, 1111.0: 40.0, 1113.0: 30.0, 1117.0: 201.0, 1115.0: 212.0, 1119.0: 31.0, 1121.0: 159.0, 1123.0: 138.0, 1125.0: 401.0, 1127.0: 239.0, 1129.0: 31.0, 1131.0: 25.0, 1133.0: 62.0, 2013.0: 0.0, 2016.0: 0.0, 2020.0: 

In [388]:
## subset last last 15 days
covid_last15 = covid[(covid['dt']>date_N1_days_ago) & (covid['dt']<= data_date_dt)].copy()
covid_last15['dt_time_delta'] = covid_last15['dt']-data_date_dt

In [389]:
## calculate new daily cases

## sort values by county and date
covid_last15.sort_values(by=['fips','dt'],inplace=True)
## remove data with 'unknown' counties
covid_last15 = covid_last15[covid_last15['fips'].notnull()].copy()

## calculate daily difference in number of cases
covid_last15['new_cases'] = covid_last15.groupby('fips')['cases'].transform(lambda x: x.diff())
## set negative new cases to zero, this can occuer due to the disperacy in the data

covid_last15.loc[covid_last15.new_cases < 1e-6, 'new_cases'] = 0
covid_last15.sort_index(inplace = True)

In [390]:
covid_last15[covid_last15.county == 'New Haven'].head(50)

,date,county,state,fips,cases,deaths,dt,dt_time_delta,new_cases
998641,2021-02-05,New Haven,Connecticut,9009.0,65488,1820.0,2021-02-05,-14 days,NaN
1001887,2021-02-06,New Haven,Connecticut,9009.0,65488,1820.0,2021-02-06,-13 days,0.0
1005133,2021-02-07,New Haven,Connecticut,9009.0,65488,1820.0,2021-02-07,-12 days,0.0
1008379,2021-02-08,New Haven,Connecticut,9009.0,66619,1835.0,2021-02-08,-11 days,1131.0
1011625,2021-02-09,New Haven,Connecticut,9009.0,66828,1839.0,2021-02-09,-10 days,209.0
1014870,2021-02-10,New Haven,Connecticut,9009.0,67075,1848.0,2021-02-10,-9 days,247.0
1018115,2021-02-11,New Haven,Connecticut,9009.0,67380,1852.0,2021-02-11,-8 days,305.0
1021360,2021-02-12,New Haven,Connecticut,9009.0,67586,1859.0,2021-02-12,-7 days,206.0
1024605,2021-02-13,New Haven,Connecticut,9009.0,67586,1859.0,2021-02-13,-6 days,0.0
1027850,2021-02-14,New Haven,Connecticut,9009.0,67586,1859.0,2021-02-14,-5 days,0.0


In [391]:
## select just last 14 days now that we have daily new cases with 15th day as baseline
covid_last14 = covid_last15[(covid_last15['dt'] > date_N_days_ago) & (covid_last15['dt'] <= data_date_dt)].copy()


In [392]:
## group by FIPS to get case load and follow up demand values for each county
covid_last14_stats = covid_last14.groupby(['fips'])['new_cases'].sum().reset_index(name ='total_cases')

covid_last14_stats[covid_last14_stats.fips == 9009].head(5)

,fips,total_cases
310,9009.0,3465.0


In [393]:
covid.tail(50)

,date,county,state,fips,cases,deaths,dt
304,12/29/2020,Richmond,New York,36085.0,584,9.0,2020-12-29
305,12/30/2020,Richmond,New York,36085.0,462,4.0,2020-12-30
306,12/31/2020,Richmond,New York,36085.0,386,3.0,2020-12-31
307,01/01/2021,Richmond,New York,36085.0,122,3.0,2021-01-01
308,01/02/2021,Richmond,New York,36085.0,508,8.0,2021-01-02
309,01/03/2021,Richmond,New York,36085.0,396,5.0,2021-01-03
310,01/04/2021,Richmond,New York,36085.0,654,8.0,2021-01-04
311,01/05/2021,Richmond,New York,36085.0,515,5.0,2021-01-05
312,01/06/2021,Richmond,New York,36085.0,523,6.0,2021-01-06
313,01/07/2021,Richmond,New York,36085.0,452,7.0,2021-01-07


In [394]:
## group by FIPS to get case load and follow up demand values for each county
#covid_death = covid.groupby(['fips'])['deaths'].sum().reset_index(name ='total_deaths')
#covid_death.head(250)

In [395]:
## group by FIPS to get case load each month and follow up demand values for each county
covid['month'] = pd.to_datetime(covid['date'])
covid['year'] = pd.to_datetime(covid['date'])


In [396]:
#covid_last14_stats_montly = covid.groupby(['fips', covid.month.dt.month, covid.year.dt.year])['deaths'].sum().reset_index() 

#covid_last14_stats_montly['cumulative_death'] = covid_last14_stats_montly.groupby(['fips'])['deaths'].cumsum(axis = 0) 
#covid_last14_stats_montly = covid_last14_stats_montly.groupby(['fips', covid_last14_stats_montly.month,  covid_last14_stats_montly.year])['deaths'].cumsum()

#covid_last14_stats_montly.head(10)

In [397]:
# adding population information from CDC svi dataset
covid_last14_stats = covid_last14_stats.reset_index()
covid_last14_stats['fips'] = covid_last14_stats['fips'].astype(int)
svi_county['FIPS'] = svi_county['FIPS'].astype(int)
covid_last14_stats = pd.merge(left = covid_last14_stats, right = svi_county[['E_TOTPOP','FIPS', 'STATE']], how = 'right', right_on = 'FIPS', left_on = 'fips' )
covid_last14_stats.fillna(0 , inplace=True)

In [398]:

covid_last14_stats.tail(5)

,index,fips,total_cases,E_TOTPOP,FIPS,STATE
3215,3219.0,72153.0,12.0,36439,72153,PUERTO RICO
3216,0.0,0.0,0.0,890,2060,ALASKA
3217,0.0,0.0,0.0,2132,2105,ALASKA
3218,0.0,0.0,0.0,689,2282,ALASKA
3219,0.0,0.0,0.0,1375,2164,ALASKA


In [399]:
now = pd.to_datetime("now")

print (now)
m_now = now.month
print (m_now)

y_now = now.year

2021-02-19 19:46:53.177611
2


In [400]:
#one_month_lag_cumulative_death =  covid_last14_stats_montly[(covid_last14_stats_montly['month'] == m_now-1) & (covid_last14_stats_montly['year'] == y_now)]

# Create a dictionary for the last month COVID deaths in each county
#one_month_lag_death = dict(zip(one_month_lag_cumulative_death.fips, one_month_lag_cumulative_death.cumulative_death))

#one_month_lag_cumulative_death.head(5)


In [401]:
#two_month_lag_cumulative_death =  covid_last14_stats_montly[(covid_last14_stats_montly['month'] == m_now-2) & (covid_last14_stats_montly['year'] == y_now)]

# Create a dictionary for the last month COVID deaths in each county
#two_month_lag_death = dict(zip(two_month_lag_cumulative_death.fips, two_month_lag_cumulative_death.cumulative_death))


#two_month_lag_cumulative_death.head(5)

In [402]:
#three_month_lag_cumulative_death =  covid_last14_stats_montly[(covid_last14_stats_montly['month'] == m_now - 3) & (covid_last14_stats_montly['year'] == y_now)]


# Create a dictionary for the 3 last month COVID deaths in each county
#three_month_lag_death = dict(zip(three_month_lag_cumulative_death.fips, three_month_lag_cumulative_death.cumulative_death))

#three_month_lag_cumulative_death.head(5)

#print (three_month_lag_death)

In [403]:

# Create a dictionary for the states of the given the county FIPS
county_of_states = dict(zip(svi_county.FIPS, svi_county.STATE))

# Create a dictionary for the name of the given the county FIPS
county_name = dict(zip(svi_county.FIPS, svi_county.COUNTY))

# Create the list for county FIPS, we consider counties as analogy to the center for community health workers
location = svi_county.FIPS.tolist() #[k for k in SVI_county] #[9001, 9003, 9005, 9007, 9009, 9011, 9013, 9015]#[k for k in SVI_county]




In [404]:
for j in County_covid_death:
       
    print (j, County_covid_death[j])

1001.0 84.0
1003.0 254.0
1005.0 49.0
1007.0 57.0
1009.0 121.0
1011.0 33.0
1013.0 64.0
1015.0 265.0
1017.0 98.0
1019.0 37.0
1021.0 92.0
1023.0 23.0
1025.0 48.0
1027.0 53.0
1029.0 35.0
1031.0 91.0
1033.0 117.0
1035.0 23.0
1037.0 22.0
1039.0 98.0
1041.0 51.0
1043.0 171.0
1045.0 105.0
1047.0 134.0
1049.0 170.0
1051.0 171.0
1053.0 60.0
1055.0 300.0
1057.0 53.0
1059.0 75.0
1061.0 67.0
1063.0 32.0
1065.0 62.0
1067.0 40.0
1069.0 251.0
1071.0 95.0
1073.0 1319.0
1075.0 32.0
1077.0 205.0
1079.0 85.0
1081.0 141.0
1083.0 127.0
1085.0 44.0
1087.0 42.0
1089.0 422.0
1091.0 52.0
1093.0 92.0
1095.0 195.0
1097.0 697.0
1099.0 37.0
1101.0 476.0
1103.0 239.0
1105.0 27.0
1107.0 52.0
1109.0 70.0
1111.0 40.0
1113.0 30.0
1117.0 201.0
1115.0 212.0
1119.0 31.0
1121.0 159.0
1123.0 138.0
1125.0 401.0
1127.0 239.0
1129.0 31.0
1131.0 25.0
1133.0 62.0
2013.0 0.0
2016.0 0.0
2020.0 151.0
2050.0 18.0
2997.0 0.0
2068.0 0.0
2070.0 1.0
2090.0 26.0
2100.0 0.0
2110.0 3.0
2122.0 19.0
2130.0 2.0
2150.0 5.0
2158.0 3.0
2170.0 33.

21067.0 219.0
21069.0 14.0
21071.0 28.0
21073.0 28.0
21075.0 6.0
21077.0 19.0
21079.0 8.0
21081.0 12.0
21083.0 70.0
21085.0 45.0
21087.0 13.0
21089.0 23.0
21091.0 15.0
21093.0 86.0
21095.0 67.0
21097.0 9.0
21099.0 33.0
21101.0 57.0
21103.0 7.0
21105.0 7.0
21107.0 128.0
21109.0 29.0
21111.0 882.0
21113.0 49.0
21115.0 18.0
21117.0 122.0
21119.0 15.0
21121.0 11.0
21123.0 11.0
21125.0 23.0
21127.0 17.0
21129.0 19.0
21131.0 2.0
21133.0 18.0
21135.0 30.0
21137.0 32.0
21139.0 16.0
21141.0 51.0
21143.0 13.0
21151.0 79.0
21153.0 1.0
21155.0 23.0
21157.0 46.0
21159.0 11.0
21161.0 27.0
21145.0 63.0
21147.0 22.0
21149.0 25.0
21163.0 13.0
21165.0 11.0
21167.0 19.0
21169.0 19.0
21171.0 37.0
21173.0 23.0
21175.0 1.0
21177.0 52.0
21179.0 31.0
21181.0 6.0
21183.0 47.0
21185.0 69.0
21187.0 7.0
21189.0 10.0
21191.0 1.0
21193.0 32.0
21195.0 51.0
21197.0 1.0
21199.0 84.0
21201.0 12.0
21203.0 13.0
21205.0 10.0
21207.0 39.0
21209.0 26.0
21211.0 44.0
21213.0 28.0
21215.0 11.0
21217.0 31.0
21219.0 24.0
21221.0

45065.0 23.0
45071.0 92.0
45073.0 118.0
45075.0 208.0
45077.0 254.0
45079.0 477.0
45081.0 43.0
45083.0 663.0
45085.0 163.0
45087.0 71.0
45089.0 84.0
45091.0 304.0
46003.0 15.0
46005.0 39.0
46007.0 9.0
46009.0 25.0
46011.0 35.0
46013.0 84.0
46015.0 9.0
46017.0 13.0
46019.0 20.0
46021.0 4.0
46023.0 18.0
46025.0 4.0
46027.0 15.0
46029.0 76.0
46031.0 12.0
46033.0 12.0
46035.0 61.0
46037.0 28.0
46039.0 8.0
46041.0 22.0
46043.0 9.0
46045.0 12.0
46047.0 15.0
46049.0 13.0
46051.0 37.0
46053.0 27.0
46055.0 9.0
46057.0 38.0
46059.0 6.0
46061.0 4.0
46063.0 1.0
46065.0 34.0
46067.0 24.0
46069.0 1.0
46071.0 14.0
46073.0 16.0
46075.0 0.0
46077.0 14.0
46079.0 17.0
46081.0 44.0
46083.0 76.0
46085.0 10.0
46091.0 5.0
46087.0 24.0
46089.0 4.0
46093.0 31.0
46095.0 2.0
46097.0 9.0
46099.0 326.0
46101.0 16.0
46102.0 47.0
46103.0 183.0
46105.0 13.0
46107.0 3.0
46109.0 35.0
46111.0 3.0
46115.0 25.0
46117.0 2.0
46119.0 3.0
46121.0 28.0
46123.0 16.0
46125.0 51.0
46127.0 39.0
46129.0 15.0
46135.0 28.0
46137.0 9.

In [405]:
K_c.head()

,fips,cases,deaths
0,36047.0,10568,260


In [406]:
K = dict(zip(K_c.fips, K_c.cases))
Q = dict(zip(Q_c.fips, Q_c.cases))
B = dict(zip(B_c.fips, B_c.cases))
M = dict(zip(M_c.fips, M_c.cases))
R  = dict(zip(R_c.fips, R_c.cases))

In [407]:
print (K)

{36047.0: 10568}


In [408]:
# Create a dictionary for the county and covid cases
covid_cases_county_ny_times = dict(zip(covid_last14_stats.fips, covid_last14_stats.total_cases))
COVID_14days = {}

for j in location:
    if j in covid_cases_county_ny_times:
        COVID_14days[j] = covid_cases_county_ny_times[j] 
        print(j, covid_cases_county_ny_times[j])
        

        
    else:
        COVID_14days[j] = 0
        
        
    if j not in County_covid_death:
        print ('j',j)
        County_covid_death[j] = 0

#print (three_month_lag_death)
#print (two_month_lag_covid_death)
#print (one_month_lag_covid_death)
#for j in location:
#    if j not in three_month_lag_death:        
#        three_month_lag_death[j] = 0
        
#    if j not in two_month_lag_death:        
#        two_month_lag_death[j] = 0
        
#    if j not in one_month_lag_death:        
#        one_month_lag_death[j] = 0
        
    #if j not in County_covid_death:
    #    County_covid_death[j] = 0

35039 101.0
1001 247.0
1009 139.0
1013 77.0
1015 508.0
1017 99.0
1031 143.0
1033 117.0
1039 178.0
1043 116.0
1045 190.0
1051 396.0
1055 263.0
1067 59.0
1069 311.0
1071 83.0
1077 176.0
1079 79.0
1083 357.0
1089 1268.0
1095 207.0
1097 1386.0
1103 274.0
1111 50.0
1113 143.0
1115 295.0
1117 754.0
1121 253.0
2261 11.0
4021 1559.0
5009 126.0
5011 39.0
5033 221.0
5037 34.0
5045 373.0
5047 64.0
5051 541.0
5053 58.0
5063 89.0
5083 104.0
5085 414.0
5087 23.0
5115 245.0
5117 30.0
5121 19.0
5125 440.0
5131 468.0
5145 236.0
6007 384.0
6017 357.0
6023 146.0
6027 113.0
6061 678.0
6079 816.0
6089 237.0
6093 97.0
8019 8.0
8039 55.0
8047 11.0
8051 100.0
8057 4.0
8065 61.0
8067 125.0
8071 47.0
8075 48.0
8077 494.0
8083 31.0
8085 110.0
8093 25.0
8099 12.0
8101 433.0
8103 33.0
8119 38.0
8121 14.0
9005 534.0
9007 498.0
9011 960.0
9015 405.0
10001 744.0
12019 680.0
12033 1213.0
12089 354.0
12101 1756.0
12109 850.0
12113 733.0
13015 795.0
13039 222.0
13045 411.0
13047 310.0
13057 1320.0
13073 746.0
13077 637.

55065 47.0
55067 23.0
55069 46.0
55071 167.0
55073 163.0
55075 44.0
55077 21.0
55081 109.0
55083 76.0
55087 458.0
55089 157.0
55091 17.0
55093 101.0
55095 150.0
55097 161.0
55101 266.0
55103 19.0
55105 372.0
55107 7.0
55109 166.0
55111 85.0
55115 45.0
55117 239.0
55119 26.0
55121 68.0
55123 39.0
55127 166.0
55131 245.0
55135 75.0
55137 57.0
55141 162.0
56003 43.0
56007 88.0
56009 40.0
56011 0.0
56013 205.0
56015 15.0
56019 2.0
56021 91.0
56025 78.0
56029 42.0
56031 14.0
56033 54.0
56035 7.0
56043 5.0
56045 6.0
1003 756.0
1019 27.0
1021 172.0
1027 62.0
1041 75.0
1049 146.0
1057 74.0
1061 49.0
1073 1800.0
1075 23.0
1081 532.0
1093 74.0
1109 107.0
1123 146.0
1125 657.0
2068 1.0
2090 146.0
2110 17.0
2122 48.0
2195 2.0
2220 2.0
2230 3.0
2275 0.0
4003 293.0
4019 3650.0
4025 388.0
5003 55.0
5005 53.0
5007 834.0
5015 59.0
5019 71.0
5023 50.0
5025 44.0
5029 72.0
5031 248.0
5043 42.0
5055 119.0
5059 146.0
5061 63.0
5071 103.0
5073 20.0
5075 43.0
5081 35.0
5091 170.0
5103 80.0
5105 69.0
5109 40.0

54081 266.0
54093 17.0
55001 45.0
55007 18.0
55013 55.0
55019 40.0
55025 1117.0
55029 34.0
55033 90.0
55037 2.0
55051 46.0
55059 371.0
55063 244.0
55085 82.0
55099 29.0
55113 46.0
55125 75.0
55129 30.0
55133 813.0
55139 324.0
56001 53.0
56005 16.0
56017 2.0
56023 36.0
56027 0.0
56037 87.0
56041 41.0
1007 75.0
1037 114.0
1053 115.0
1101 861.0
1107 69.0
1127 174.0
1129 50.0
2020 550.0
2100 1.0
2130 31.0
4005 573.0
4011 15.0
5001 84.0
5013 24.0
5021 26.0
5027 80.0
5119 1384.0
5129 11.0
5135 35.0
5141 29.0
5149 130.0
6005 127.0
6015 60.0
6051 30.0
6055 292.0
6087 604.0
6097 819.0
6103 184.0
6111 3639.0
8033 5.0
8037 272.0
8045 100.0
8053 5.0
8055 6.0
8073 26.0
8087 30.0
8109 18.0
8111 1.0
8117 195.0
8125 23.0
9001 3390.0
9009 3465.0
12003 137.0
12007 99.0
12023 206.0
12055 380.0
12057 4862.0
12061 606.0
12065 54.0
12081 1533.0
12085 408.0
12087 178.0
12099 6101.0
12115 978.0
12129 111.0
13007 4.0
13011 49.0
13023 37.0
13029 220.0
13031 293.0
13053 219.0
13075 56.0
13079 19.0
13111 101.0
13

6029 4643.0
6031 637.0
6047 1251.0
12027 179.0
12039 168.0
12059 44.0
13003 12.0
13049 33.0
13071 149.0
13093 19.0
13141 27.0
13309 14.0
16033 0.0
21025 39.0
21051 84.0
21131 45.0
21189 35.0
21193 128.0
22027 52.0
22039 109.0
28001 61.0
28011 140.0
28027 89.0
28063 10.0
28079 44.0
28083 83.0
28119 21.0
28143 34.0
29155 15.0
30003 41.0
37155 518.0
38085 2.0
42101 4194.0
45067 209.0
46007 3.0
46041 9.0
46071 8.0
48109 11.0
48141 4709.0
48163 70.0
48261 2.0
51595 28.0
54059 104.0
1005 65.0
1063 36.0
1105 34.0
2050 107.0
4001 384.0
5077 20.0
6019 3429.0
6107 2328.0
12047 27.0
13193 38.0
13239 3.0
13259 79.0
21119 83.0
21147 113.0
21159 23.0
21237 14.0
22035 8.0
28051 27.0
28053 24.0
28103 27.0
28133 95.0
28163 103.0
30085 31.0
35006 108.0
35029 67.0
46017 1.0
46031 2.0
47095 24.0
48061 778.0
48215 4643.0
48377 83.0
48445 15.0
48479 1601.0
48489 181.0
53001 57.0
54047 47.0
1085 38.0
2158 39.0
6025 406.0
12049 116.0
12051 142.0
13037 39.0
13265 3.0
22065 22.0
28055 3.0
28123 98.0
28135 36.0


In [409]:
for j in location: 
    if j in K:
        COVID_14days[j] = K[j]
        print (j, COVID_14days[j], K[j])

    if j in Q:
        COVID_14days[j] = Q[j]
        print (j, COVID_14days[j])
        
    if j in B:
        COVID_14days[j] = B[j]
        print (j, COVID_14days[j])
        
    if j in M:
        COVID_14days[j] = M[j]
        print (j, COVID_14days[j])

    if j in R:
        COVID_14days[j] = R[j]
        print (j, COVID_14days[j])

36085 2287
36061 5023
36047 10568 10568
36081 10177
36005 7768


In [410]:
print (covid_cases_county_ny_times)

{1001.0: 247.0, 1003.0: 756.0, 1005.0: 65.0, 1007.0: 75.0, 1009.0: 139.0, 1011.0: 34.0, 1013.0: 77.0, 1015.0: 508.0, 1017.0: 99.0, 1019.0: 27.0, 1021.0: 172.0, 1023.0: 7.0, 1025.0: 99.0, 1027.0: 62.0, 1029.0: 30.0, 1031.0: 143.0, 1033.0: 117.0, 1035.0: 37.0, 1037.0: 114.0, 1039.0: 178.0, 1041.0: 75.0, 1043.0: 116.0, 1045.0: 190.0, 1047.0: 69.0, 1049.0: 146.0, 1051.0: 396.0, 1053.0: 115.0, 1055.0: 263.0, 1057.0: 74.0, 1059.0: 63.0, 1061.0: 49.0, 1063.0: 36.0, 1065.0: 43.0, 1067.0: 59.0, 1069.0: 311.0, 1071.0: 83.0, 1073.0: 1800.0, 1075.0: 23.0, 1077.0: 176.0, 1079.0: 79.0, 1081.0: 532.0, 1083.0: 357.0, 1085.0: 38.0, 1087.0: 69.0, 1089.0: 1268.0, 1091.0: 74.0, 1093.0: 74.0, 1095.0: 207.0, 1097.0: 1386.0, 1099.0: 33.0, 1101.0: 861.0, 1103.0: 274.0, 1105.0: 34.0, 1107.0: 69.0, 1109.0: 107.0, 1111.0: 50.0, 1113.0: 143.0, 1115.0: 295.0, 1117.0: 754.0, 1119.0: 24.0, 1121.0: 253.0, 1123.0: 146.0, 1125.0: 657.0, 1127.0: 174.0, 1129.0: 50.0, 1131.0: 37.0, 1133.0: 41.0, 2013.0: 96.0, 2016.0: 107.

In [411]:
#Parameters
pro_c_s = [(i,county_of_states[i]) for i in location ]
cartesian_pro_county_state = gp.tuplelist(pro_c_s)


In [412]:
df = covid_last14_stats
df['fips'] = df['fips'].astype(int)

In [413]:
###############################################################################################
######################## END calculating different types of vulnerabilities ###################

Since we allocate CHW proportional to the county values of certain vulnaribilites within state, we need a few function to help us with the calculations. 



In [414]:
# This function return the value for the state for the given dictionary

# More specifically sum upt the values for the counties of each state

def total_state(dict_1):
    state_dict = {}
    for s in State:
        state_dict [s] = sum(float(dict_1[j]) for (j,s) in cartesian_pro_county_state.select('*', s) if j in dict_1) 
        #print (state_dict [s],s)
    return state_dict



In [415]:
# Calculte the population per state by summing up the population in each county in the state
State_pop = total_state(population_county)

In [416]:


# This function returns the ratio of the dict value for county and state of the county
def Proportional(county_level, state_level):
    
    prop = {}
       
    for (j,s) in cartesian_pro_county_state:
        if state_level[s] >= 1e-6 and j in county_level:
            prop[j] = (float(county_level[j])/float(state_level[s]))    
        else:
            prop[j] = 0
                
    return prop

In [417]:
print (ACI_total)
print (ACI_total[35039])

{1001: 9804, 1003: 33019, 1005: 7332, 1007: 4860, 1009: 8954, 1011: 3100, 1013: 5004, 1015: 25387, 1017: 7391, 1019: 5162, 1021: 9558, 1023: 3433, 1025: 6557, 1027: 3375, 1029: 2964, 1031: 9652, 1033: 9471, 1035: 2710, 1037: 2234, 1039: 6987, 1041: 3339, 1043: 15529, 1045: 10717, 1047: 13248, 1049: 17983, 1051: 12740, 1053: 6301, 1055: 21800, 1057: 4277, 1059: 6775, 1061: 6484, 1063: 2858, 1065: 4977, 1067: 3485, 1069: 23013, 1071: 11372, 1073: 125514, 1075: 3015, 1077: 15659, 1079: 6971, 1081: 23354, 1083: 15392, 1085: 3457, 1087: 5200, 1089: 48472, 1091: 4803, 1093: 6723, 1095: 21817, 1097: 93203, 1099: 5664, 1101: 52425, 1103: 21849, 1105: 3094, 1107: 4752, 1109: 6901, 1111: 4507, 1113: 13627, 1115: 13737, 1117: 19953, 1119: 4144, 1121: 19136, 1123: 9809, 1125: 34475, 1127: 13645, 1129: 4400, 1131: 4082, 1133: 5781, 2013: 451, 2016: 439, 2020: 51594, 2050: 9063, 2060: 120, 2068: 145, 2070: 2082, 2090: 13016, 2100: 731, 2105: 512, 2110: 4856, 2122: 12129, 2130: 2919, 2150: 2451, 2158

In [418]:
#ACI
ACI_State = total_state(ACI_total)

In [419]:
# Create dicts for the variables of SVI
E_POV = dict(zip(svi_county.FIPS, svi_county.EP_POV))
E_UNEMP = dict(zip(svi_county.FIPS, svi_county.EP_UNEMP))
E_PCI = dict(zip(svi_county.FIPS, svi_county.EP_PCI))
E_NOHSDP = dict(zip(svi_county.FIPS, svi_county.EP_NOHSDP))
E_AGE65 = dict(zip(svi_county.FIPS, svi_county.EP_AGE65))
E_AGE17 = dict(zip(svi_county.FIPS, svi_county.EP_AGE17))
E_DISABL = dict(zip(svi_county.FIPS, svi_county.EP_DISABL))
E_SNGPNT = dict(zip(svi_county.FIPS, svi_county.EP_SNGPNT))
E_MINRTY = dict(zip(svi_county.FIPS, svi_county.EP_MINRTY))
E_LIMENG = dict(zip(svi_county.FIPS, svi_county.EP_LIMENG))
E_MUNIT = dict(zip(svi_county.FIPS, svi_county.EP_MUNIT))
E_MOBILE = dict(zip(svi_county.FIPS, svi_county.EP_MOBILE))
E_CROWD = dict(zip(svi_county.FIPS, svi_county.EP_CROWD))
E_NOVEH = dict(zip(svi_county.FIPS, svi_county.EP_NOVEH))
E_GROUPQ = dict(zip(svi_county.FIPS, svi_county.EP_GROUPQ))


# Calculate the state value for the SVI variables
E_POV_State = total_state(E_POV)
E_UNEMP_State = total_state(E_UNEMP) 
E_PCI_State = total_state(E_PCI)
E_NOHSDP_State = total_state(E_NOHSDP) 
E_AGE65_State = total_state(E_AGE65)
E_AGE17_State = total_state(E_AGE17)
E_DISABL_State = total_state(E_DISABL)
E_SNGPNT_State = total_state(E_SNGPNT)
E_MINRTY_State = total_state(E_MINRTY)
E_LIMENG_State = total_state(E_LIMENG)
E_MUNIT_State = total_state(E_MUNIT)
E_MOBILE_State = total_state(E_MOBILE)
E_CROWD_State = total_state(E_CROWD)
E_NOVEH_State = total_state(E_NOVEH)
E_GROUPQ_State = total_state(E_GROUPQ)



# Calculate the proportinal values for the SVI variables
E_POV_Prop = Proportional(E_POV, E_POV_State )
E_UNEMP_Prop = Proportional(E_UNEMP, E_UNEMP_State ) 
E_PCI_Prop = Proportional(E_PCI, E_PCI_State )
E_NOHSDP_Prop = Proportional(E_NOHSDP, E_NOHSDP_State ) 
E_AGE65_Prop = Proportional(E_AGE65, E_AGE65_State )
E_AGE17_Prop = Proportional(E_AGE17, E_AGE17_State )
E_DISABL_Prop = Proportional(E_DISABL, E_DISABL_State )
E_SNGPNT_Prop = Proportional(E_SNGPNT, E_SNGPNT_State )
E_MINRTY_Prop = Proportional(E_MINRTY, E_MINRTY_State )
E_LIMENG_Prop = Proportional(E_LIMENG, E_LIMENG_State )
E_MUNIT_Prop = Proportional(E_MUNIT, E_MUNIT_State )
E_MOBILE_Prop = Proportional(E_MOBILE, E_MOBILE_State )
E_CROWD_Prop = Proportional(E_CROWD, E_CROWD_State )
E_NOVEH_Prop = Proportional(E_NOVEH, E_NOVEH_State )
E_GROUPQ_Prop = Proportional(E_GROUPQ, E_GROUPQ_State )

ACI_Prop = Proportional(ACI_total, ACI_State)


# SVI calculation 

We calculate the ratio of county value to state value by population for each SVI variables (we use EP-estimate percentage- values in the CDC data set), then we take the average of all 15 SVI variables. 

Let SVI variable set be K, where  

K = { Below Poverty, Unemployed, Income, No High School Diploma, Aged 65 or Older, Aged 17 or Younger, Civilian with a Disability, Single-Parent Households, Minority, Speaks English “Less than Well”, Multi-Unit Structures, Mobile Homes, Crowding, No Vehicle, Group Quarters }

We will use these variables in a county base and state base. While County base values are exactly same as the estimated values for these variables in the CDC website, to calculate the state base, we simply sum the county values for all of the counties in each state for each variable. Then we use the following formula to calculate the SVI value for each county.

Let $S$ is the set of states and $j$ is a county in the state $s$, where $s \in S$, $c^k_j$ SVI variable $k \in K$ value for county j, and $c_s$ SVI variable value for state s.

$SVI_j = \frac{1}{15}\sum_{k \in K} \frac{c^k_j}{c^k_s}$


In [420]:

from collections import Counter
# Sum all SVI variable values for each county
SVI_county_sum = dict(Counter(E_POV_Prop) + Counter(E_UNEMP_Prop) + Counter(E_PCI_Prop) + Counter(E_NOHSDP_Prop) + Counter(E_AGE65_Prop) + Counter(E_AGE17_Prop) + Counter(E_DISABL_Prop) + Counter(E_SNGPNT_Prop) + Counter(E_MINRTY_Prop) + Counter(E_LIMENG_Prop) + Counter(E_MUNIT_Prop) + Counter(E_MOBILE_Prop) + Counter(E_CROWD_Prop) + Counter(E_NOVEH_Prop) + Counter(E_GROUPQ_Prop))

# Divide the sum of all SVI variable values
SVI_county = {j: SVI_county_sum[j]/15 for j in SVI_county_sum }


# Proportional Allocation

We consider allocating 1 million CHW over the states proportional to Medicaid enrollment in each state. Further, we allocate CHW to counties in each state proportional to different county vulnerability criterias as follow.

- MEDICAID
- SVI
- YPLL
- UNEMPLOYMENT
- LAST 14 DAYS COVID CASES
- LAST 14 DAYS COVID CASES / POP
- COVID DEATHS / POP

To calculate the total number of allocated CHW to per county according to these vulnerability criterias, we define the following function called "Proportional_allocation", in which we multiply the CHW allocated to each state with the ratio of the chosen vulnerability criteria of the county to the chosen vulnerability criteria of the state, the function return a dictionary with the counties as keys and the number of CHW allocated to each county for the chosen vulnerability criteria as values. 

In [421]:
def Proportional_allocation(county_level, state_level, state_budget):
    prop_allocate = {}
       
    for (j,s) in cartesian_pro_county_state:
        if state_level[s] >= 1e-6 and j in county_level:
            #print (j,s, county_level[j],state_level[s], state_budget[s])
            prop_allocate[j,s] = (float(county_level[j])/float(state_level[s]))*float(state_budget[s])
        
        else:
            prop_allocate[j,s] = 0
            
    
    return prop_allocate

In [422]:
Medicaid_dem = Proportional_allocation(ACI_total, ACI_State, Medicaid_state)
Medicaid_demand = {m[0]: Medicaid_dem[m] for m in Medicaid_dem}
for s in State:
#    print (s)
#    print ( ACI_State[s])
    print (s,Medicaid_state[s])
#for m in Medicaid_demand:
#    print (m, Medicaid_demand[m])

NEW MEXICO 772102
ALABAMA 957116
ALASKA 231145
ARIZONA 1839932
ARKANSAS 830467
CALIFORNIA 11847711
COLORADO 1337805
CONNECTICUT 874974
DELAWARE 239009
FLORIDA 3892552
GEORGIA 1928703
IDAHO 340742
ILLINOIS 2987496
INDIANA 1602976
IOWA 699741
KANSAS 401103
KENTUCKY 1416013
LOUISIANA 1585024
MAINE 232455
MARYLAND 1372695
MASSACHUSETTS 1616404
MICHIGAN 2439425
MINNESOTA 1085778
MISSISSIPPI 632427
MISSOURI 923641
MONTANA 257006
NEBRASKA 254159
NEVADA 685073
NEW HAMPSHIRE 193436
NEW JERSEY 1759653
NEW YORK 6263164
NORTH CAROLINA 1851558
NORTH DAKOTA 96757
OHIO 2788134
OKLAHOMA 797220
OREGON 1053931
PENNSYLVANIA 3069309
RHODE ISLAND 305208
SOUTH CAROLINA 1048276
SOUTH DAKOTA 114059
TENNESSEE 1489536
TEXAS 4457644
UTAH 338812
VERMONT 161049
VIRGINIA 1497770
WASHINGTON 1790395
WEST VIRGINIA 521290
WISCONSIN 1112844
WYOMING 59302
HAWAII 351337
DISTRICT OF COLUMBIA 250860
PUERTO RICO 1622194


In [423]:
#print ('Med', Medicaid_demand[35039])
#print ('Med', Medicaid_demand[4017], 'Pop', population_county[4017], 'Med_capita', 100000*(Medicaid_demand[4017]/population_county[4017]))

County_covid_death[72049]

nan

In [424]:
#Further we create additional vulnerability values by considering SVI, YPLL, Unemployment, COVID, COVID_capita, COVID_death and COVID_death capita and the number of Medicaid enrolles in each county together

Covid_capita = {j: 100000*(COVID_14days[j]/population_county[j]) for j in location}

Covid_death_capita = {j: 100000*(County_covid_death[j]/population_county[j]) for j in location}

Medicaid_capita = {j: 100000*(Medicaid_demand[j]/population_county[j]) for j in location}

Unemployment_capita = dict(zip(df_unemp.FIPS, df_unemp.Unemployed_Rate))

#Three_month_lag_covid_death_capita = {j: 100000*(three_month_lag_death[j]/population_county[j]) for j in location}

#Two_month_lag_covid_death_capita = {j: 100000*(two_month_lag_death[j]/population_county[j]) for j in location}

#One_month_lag_covid_death_capita = {j: 100000*(one_month_lag_death[j]/population_county[j]) for j in location}

In [425]:
for j in location:
    print (j, County_covid_death[j], population_county[j], Covid_death_capita[j])

35039 62.0 38921 159.29703758896227
1001 84.0 55869 150.3517156204693
1009 121.0 57826 209.24843496005255
1013 64.0 19448 329.0826820238585
1015 265.0 113605 233.2643809691475
1017 98.0 33254 294.7013893065496
1031 91.0 52342 173.856558786443
1033 117.0 55241 211.79920711066057
1039 98.0 37049 264.51456179653974
1043 171.0 83768 204.13523063699742
1045 105.0 49172 213.53615878955506
1051 171.0 81209 210.56779421000138
1055 300.0 102268 293.3468924785857
1067 40.0 17205 232.49055507120025
1069 251.0 105882 237.05634574337472
1071 95.0 51626 184.01580598923024
1077 205.0 92729 221.0743133216146
1079 85.0 32924 258.17033167294375
1083 127.0 98915 128.39306475256532
1089 422.0 372909 113.16433768023835
1095 195.0 96774 201.50040300080602
1097 697.0 413210 168.67936400377533
1103 239.0 119679 199.70086648451274
1111 40.0 22722 176.04084147522227
1113 30.0 57961 51.75894135711944
1115 212.0 89512 236.83975332916256
1117 201.0 217702 92.32804475843125
1121 159.0 79978 198.80467128460324
2261 

28089 189.0 106272 177.84552845528455
28093 92.0 35294 260.66753555845185
28095 125.0 35252 354.58981050720524
28109 126.0 55535 226.88394706041237
28121 257.0 155271 165.517063714409
28129 29.0 15916 182.20658456898718
28159 74.0 17955 412.1414647730437
28161 36.0 12108 297.3240832507433
29003 17.0 17712 95.98012646793134
29007 57.0 25388 224.51551914290215
29011 12.0 11754 102.09290454313425
29013 22.0 16172 136.0375958446698
29021 162.0 87364 185.4310700059521
29025 10.0 9020 110.86474501108647
29031 132.0 78871 167.36189473951134
29033 22.0 8679 253.48542458808618
29037 84.0 105780 79.41009642654566
29039 17.0 14349 118.47515506307059
29041 6.0 7426 80.79719903043362
29043 75.0 88595 84.65489023082567
29045 6.0 6797 88.27423863469177
29047 141.0 249948 56.41173364059725
29049 63.0 20387 309.02045421101684
29051 114.0 76745 148.54387908007038
29057 30.0 7561 396.77291363576245
29059 22.0 16878 130.347197535253
29061 11.0 8278 132.88233872916163
29071 153.0 103967 147.16208027547202


5031 174.0 110332 157.7058333031215
5043 35.0 18219 192.10714089686593
5055 74.0 45325 163.26530612244898
5059 68.0 33771 201.35619318350064
5061 24.0 13202 181.79063778215422
5071 31.0 26578 116.63782075400707
5073 7.0 6624 105.67632850241544
5075 41.0 16406 249.90857003535294
5081 40.0 12259 326.29088832694345
5091 33.0 43257 76.28823080657466
5103 50.0 23382 213.83970575656485
5105 8.0 10455 76.51841224294596
5109 21.0 10718 195.93207688001496
5139 98.0 38682 253.34781035106766
5143 327.0 239187 136.71311567936382
6009 25.0 45905 54.460298442435466
6033 39.0 64386 60.572174075109494
6035 19.0 30573 62.146338272331796
6041 192.0 258826 74.18111008940369
6043 7.0 17203 40.69057722490263
6049 4.0 8841 45.243750706933604
6057 74.0 99755 74.18174527592602
6063 6.0 18807 31.9030148349019
6091 0.0 3005 0.0
6105 5.0 12285 40.700040700040695
6109 56.0 54478 102.7937883182202
8003 28.0 16233 172.488141440276
8005 687.0 656590 104.63150520111486
8007 1.0 14029 7.128091809822511
8009 4.0 3581 1

37105 65.0 61779 105.21374577121675
37111 55.0 45756 120.20281493137512
37113 20.0 35858 55.77555914998048
37117 37.0 22440 164.88413547237076
37129 139.0 234473 59.281878937020466
37135 92.0 148476 61.96287615506883
37137 9.0 12726 70.72135785007072
37143 7.0 13463 51.99435489861101
37149 25.0 20724 120.63308241652192
37173 15.0 14271 105.10826150935462
37175 26.0 34385 75.61436672967864
37179 189.0 239859 78.79629282203294
37183 518.0 1111761 46.59274790175226
37191 203.0 123131 164.8650624132022
37193 99.0 68412 144.71145413085424
37195 145.0 81801 177.25944670603047
37199 23.0 18069 127.28983341634844
38013 1.0 2115 47.28132387706856
38015 185.0 95626 193.46202915525066
38017 193.0 181923 106.0888397838646
38019 5.0 3762 132.90802764486975
38023 2.0 2264 88.33922261484099
38029 12.0 3241 370.25609379821043
38035 74.0 69451 106.54994168550489
38037 7.0 2274 307.82761653474057
38039 2.0 2231 89.64589870013447
38045 17.0 4046 420.16806722689074
38047 8.0 1850 432.43243243243245
38051 

5097 28.0 8986 311.59581571333183
5101 26.0 7753 335.3540564942603
5111 75.0 23528 318.76912614756884
5127 14.0 10281 136.1735239762669
5137 30.0 12506 239.88485526947068
5147 10.0 6320 158.2278481012658
6003 0.0 1129 0.0
6021 23.0 28393 81.00588173141267
6045 41.0 86749 47.26279265467037
6065 3599.0 2470546 145.67629989484107
6069 57.0 62808 90.75277034772641
6073 3135.0 3338330 93.90923006413387
6095 156.0 447643 34.849199026903136
6113 181.0 220500 82.08616780045351
8011 21.0 5577 376.54653039268425
8031 762.0 727211 104.78389353296362
8089 61.0 18278 333.73454426086005
8105 12.0 11267 106.50572468270171
9003 2246.0 891720 251.87278517920424
11001 992.0 705749 140.5598874387353
12011 2297.0 1952778 117.62729813629609
12063 140.0 46414 301.6331279355367
12071 859.0 770577 111.47490776392236
12095 1059.0 1393452 75.99831210547617
12097 416.0 375751 110.71161487261512
12133 40.0 25473 157.02901110980252
13001 60.0 18386 326.33525508539105
13009 105.0 44890 233.90510135887723
13025 27.0

4001 360.0 71887 500.7859557360858
5077 35.0 8857 395.1676639945806
6019 1349.0 999101 135.02138422441774
6107 717.0 466195 153.79830328510602
12047 21.0 14428 145.55031882450788
13193 25.0 12947 193.09492546535876
13239 1.0 2299 43.49717268377555
13259 21.0 6621 317.17263253285006
21119 15.0 14806 101.31027961637176
21147 22.0 17231 127.67686147060529
21159 11.0 11195 98.25815096025012
21237 4.0 7157 55.88933910856504
22035 22.0 6861 320.65296603993585
28051 69.0 17010 405.6437389770723
28053 26.0 8064 322.4206349206349
28103 29.0 10417 278.3910914850725
28133 84.0 25110 334.52807646356035
28163 62.0 29690 208.82452004041767
30085 52.0 11004 472.55543438749544
35006 103.0 26675 386.1293345829429
35029 73.0 23709 307.89995360411655
46017 13.0 1962 662.5891946992864
46031 12.0 4086 293.6857562408223
47095 25.0 7016 356.3283922462942
48061 1396.0 423163 329.8965174176381
48215 2575.0 868707 296.4175493002819
48377 23.0 6704 343.0787589498807
48445 54.0 12337 437.7077085190889
48479 736.0

In [426]:
County_covid_death[9009], population_county[9009], Covid_death_capita[9009]

(1885.0, 854757, 220.53051335057802)

In [427]:
total_state(Covid_death_capita) 

{'NEW MEXICO': 6105.258731140889,
 'ALABAMA': 15721.222516043397,
 'ALASKA': 570.3347371515155,
 'ARIZONA': 4073.9647872449823,
 'ARKANSAS': 15695.298181685564,
 'CALIFORNIA': 4819.002447135208,
 'COLORADO': 6941.469980999893,
 'CONNECTICUT': 1448.0928569615971,
 'DELAWARE': 444.245443983788,
 'FLORIDA': 10702.76673061081,
 'GEORGIA': 35289.76558160373,
 'IDAHO': 4402.11826326674,
 'ILLINOIS': 20970.599932609854,
 'INDIANA': 17804.073075582142,
 'IOWA': 21978.260100773427,
 'KANSAS': 16333.371933395681,
 'KENTUCKY': 14094.725945884362,
 'LOUISIANA': 15358.31882008538,
 'MAINE': 639.3209121407948,
 'MARYLAND': 3049.9087487054144,
 'MASSACHUSETTS': 2614.4393642045593,
 'MICHIGAN': 12411.618433422656,
 'MINNESOTA': 10824.49400607301,
 'MISSISSIPPI': 21877.55409704509,
 'MISSOURI': 15054.365720552245,
 'MONTANA': 9944.046543753866,
 'NEBRASKA': 12288.200037766444,
 'NEVADA': 1925.6284190997362,
 'NEW HAMPSHIRE': 691.4335055757274,
 'NEW JERSEY': 4959.28191378538,
 'NEW YORK': 8175.58418463

In [428]:
# Dictionary for the total Covid per capita for each state
Total_covid_cap = total_state(Covid_capita) 


# Dictionary for the total Covid death per capita for each state
Total_covid_death_cap = total_state(Covid_death_capita) 


# Dictionary for the total Medicaid per capita for each state
Total_medicaid_cap = total_state(Medicaid_capita)


# Dictionary for the total Unemployment per capita for each state
Total_unemployment_cap = total_state(Unemployment_capita)



In [429]:
Covid_death_capita[72049]

nan

In [430]:
# Dictionaries for the different vulnerability criteria values for states

# Dictionary for total Medicaid patient numbers for each state
Medicaid_demand_state = total_state(Medicaid_demand) 

# Dictionary for total positive COVID cases for last 14 days in each state
Covid_state = total_state(COVID_14days) 

# Dictionary for total SVI values for each state
SVI_state = total_state(SVI_county) 

# Dictionary for total YPLL values for each state
YPLL_state = total_state(YPLL) 

# Dictionary for total Unemployment numbers for each state
Unemployment_state = total_state(Unemployment)  

In [431]:
print(Medicaid_demand_state)

{'NEW MEXICO': 772102.0, 'ALABAMA': 957116.0000000001, 'ALASKA': 231145.0, 'ARIZONA': 1839932.0, 'ARKANSAS': 830467.0000000003, 'CALIFORNIA': 11847711.0, 'COLORADO': 1337805.0, 'CONNECTICUT': 874974.0, 'DELAWARE': 239009.0, 'FLORIDA': 3892551.999999999, 'GEORGIA': 1928702.999999999, 'IDAHO': 340742.0000000001, 'ILLINOIS': 2987496.0000000005, 'INDIANA': 1602975.9999999995, 'IOWA': 699741.0, 'KANSAS': 401103.0000000001, 'KENTUCKY': 1416013.0000000002, 'LOUISIANA': 1585023.9999999995, 'MAINE': 232454.99999999994, 'MARYLAND': 1372695.0, 'MASSACHUSETTS': 1616403.9999999998, 'MICHIGAN': 2439425.0000000005, 'MINNESOTA': 1085778.0000000002, 'MISSISSIPPI': 632427.0000000001, 'MISSOURI': 923640.9999999998, 'MONTANA': 257005.99999999994, 'NEBRASKA': 254159.0, 'NEVADA': 685073.0, 'NEW HAMPSHIRE': 193436.0, 'NEW JERSEY': 1759653.0, 'NEW YORK': 6263164.000000002, 'NORTH CAROLINA': 1851557.999999999, 'NORTH DAKOTA': 96756.99999999999, 'OHIO': 2788133.9999999995, 'OKLAHOMA': 797219.9999999998, 'OREGON

# 1 million CHW allocation to states

We allocate 1 million CHWs to states proportional to total Medicaid enrolles in each state.

Let's $FedCHW$ represents the number of CHW will be allocated within states by the federal government, which is 1 million in our project. $TotMed$ represents the total Medicaid enrollee numbers over the US, $Med_s$ is the total Medicaid enrollee numbers in state $s \in S$, and $CHW_s$ is the total number of CHW allocated to state $ s\in S$. 

$CHW_s = FedCHW*\frac{Med_s}{TotMed}$



In [432]:
# We consider allocation of 1 million CHW all over the US
Federal_budget_CHW = 1000000

# First, we calculate the Total Medicaid enrolles all over the US
Total_federal_need = sum(Medicaid_demand_state[s] for s in State)

# Allocate the 1 million CHWs proportional to Medicaid enrolles in each state
Medicaid_budget_state = {s: (Medicaid_demand_state[s]/Total_federal_need)*Federal_budget_CHW  for s in State}

In [433]:
for s in State:
    print (s, Medicaid_demand_state[s], Total_federal_need, Federal_budget_CHW, (Medicaid_demand_state[s]/Total_federal_need)*Federal_budget_CHW)

NEW MEXICO 772102.0 76277412.0 1000000 10122.288889402804
ALABAMA 957116.0000000001 76277412.0 1000000 12547.830018144821
ALASKA 231145.0 76277412.0 1000000 3030.3204309029256
ARIZONA 1839932.0 76277412.0 1000000 24121.583988717393
ARKANSAS 830467.0000000003 76277412.0 1000000 10887.456433367199
CALIFORNIA 11847711.0 76277412.0 1000000 155323.9771690209
COLORADO 1337805.0 76277412.0 1000000 17538.67842291241
CONNECTICUT 874974.0 76277412.0 1000000 11470.945028916292
DELAWARE 239009.0 76277412.0 1000000 3133.417793461582
FLORIDA 3892551.999999999 76277412.0 1000000 51031.516381284666
GEORGIA 1928702.999999999 76277412.0 1000000 25285.37543984842
IDAHO 340742.0000000001 76277412.0 1000000 4467.141596256571
ILLINOIS 2987496.0000000005 76277412.0 1000000 39166.195098491284
INDIANA 1602975.9999999995 76277412.0 1000000 21015.08110946396
IOWA 699741.0 76277412.0 1000000 9173.633211362756
KANSAS 401103.0000000001 76277412.0 1000000 5258.47678209114
KENTUCKY 1416013.0000000002 76277412.0 10000

In [434]:
print (Covid_state)

{'NEW MEXICO': 4948.0, 'ALABAMA': 15159.0, 'ALASKA': 1867.0, 'ARIZONA': 26141.0, 'ARKANSAS': 9900.0, 'CALIFORNIA': 117628.0, 'COLORADO': 14938.0, 'CONNECTICUT': 12454.0, 'DELAWARE': 4296.0, 'FLORIDA': 86052.0, 'GEORGIA': 43025.0, 'IDAHO': 3730.0, 'ILLINOIS': 27783.0, 'INDIANA': 17118.0, 'IOWA': 7922.0, 'KANSAS': 7433.0, 'KENTUCKY': 18581.0, 'LOUISIANA': 12272.0, 'MAINE': 2293.0, 'MARYLAND': 13386.0, 'MASSACHUSETTS': 24669.0, 'MICHIGAN': 14890.0, 'MINNESOTA': 10105.0, 'MISSISSIPPI': 9792.0, 'MISSOURI': 9879.0, 'MONTANA': 2696.0, 'NEBRASKA': 4472.0, 'NEVADA': 7264.0, 'NEW HAMPSHIRE': 4793.0, 'NEW JERSEY': 45263.0, 'NEW YORK': 87490.0, 'NORTH CAROLINA': 46698.0, 'NORTH DAKOTA': 1050.0, 'OHIO': 32950.0, 'OKLAHOMA': 16803.0, 'OREGON': 5578.0, 'PENNSYLVANIA': 43554.0, 'RHODE ISLAND': 5493.0, 'SOUTH CAROLINA': 35077.0, 'SOUTH DAKOTA': 1943.0, 'TENNESSEE': 29531.0, 'TEXAS': 105094.0, 'UTAH': 11840.0, 'VERMONT': 1569.0, 'VIRGINIA': 36721.0, 'WASHINGTON': 12107.0, 'WEST VIRGINIA': 5169.0, 'WISCO

# Proportional allocation for different vulnerability values

Let V = {Medicaid, SVI, YPLL, Unemployment, COVID, COVID_capita, COVID_death and COVID_death capita, SVI and Medicaid, YPLL and Medicaid, Unemployment and Medicaid, COVID and Medicaid, COVID_capita and Medicaid, COVID_death and Medicaid and COVID_death capita and Medicaid}. We assume $v_j$ represent the vulnerability value for county $j \in J$, while $v_s$ represent the sum of the vulnerability values for each county in the state of county j.

$Prop_{v_j} = \frac{v_j}{v_s}*CHW_s$



In [435]:
# Calling proportional allocation function for different vulnerability criterias

# Proportional allocation according to cumulative Covid death in per capita in each county
Proportional_to_covid_death_cap = Proportional_allocation(Covid_death_capita, Total_covid_death_cap,Medicaid_budget_state)

# Propportional allocation according to Medicaid enrollee number in each county
Proportional_to_medicaid = Proportional_allocation(Medicaid_demand, Medicaid_demand_state,Medicaid_budget_state )

# Proportional allocation according to Medicaid enrolles per capita in each county
Proportional_to_medicaid_cap = Proportional_allocation(Medicaid_capita, Total_medicaid_cap, Medicaid_budget_state )

# Proportional allocation according to last 14 days positive COVID cases in each county
Proportional_to_covid = Proportional_allocation(COVID_14days, Covid_state, Medicaid_budget_state)

# Proportional allocation according to SVI score in each county
Proportional_to_SVI = Proportional_allocation(SVI_county, SVI_state, Medicaid_budget_state)

# Proportional allocation according to YPLL in each county
Proportional_to_YPLL = Proportional_allocation(YPLL, YPLL_state, Medicaid_budget_state)

# Proportional allocation according to Unemployment  in each county
Proportional_to_unemployment = Proportional_allocation(Unemployment, Unemployment_state, Medicaid_budget_state)

# Proportional allocation according to Medicaid enrolles per capita in each county
Proportional_to_unemployment_cap = Proportional_allocation(Unemployment_capita, Total_unemployment_cap, Medicaid_budget_state )

# Proportional allocation according to last 14 days positive COVID cases per capita in each county
Proportional_to_covid_capita = Proportional_allocation(Covid_capita, Total_covid_cap, Medicaid_budget_state)



In [436]:
for j in Covid_death_capita:
    print (j, County_covid_death[j], population_county[j], Covid_death_capita[j])
    print(Proportional_to_covid_death_cap[j])


35039 62.0 38921 159.29703758896227


KeyError: 35039

In [ ]:
cartesian_pro_county_state

In [ ]:
Total_covid_death_cap


# Normalize values for comparison
To be able compare the different vulnerability values for each county, we normalize all vulnerability values as follows. 

Let $m_{v_s} = \min \{v_j, \text{ for county j in state s }\}$  and 
$M_{v_s} = \max \{v_j, \text{ for county j in state s }\}$.

We calculate the normalize value for each vulnerability for each county by substracting the min vulnerability in the state of the county and dividing that by the differences between max and min value of the vulnerability values in the state. Mathematical formulation for the normalization is as follows.

$N_{v_j} = \frac{v_j - m_{v_s}}{M_{v_s} - m_{v_s}}$ 

for each $v \in V$, where V = {Medicaid, SVI, YPLL, Unemployment, COVID, COVID_capita, COVID_death and COVID_death capita, SVI and Medicaid, YPLL and Medicaid, Unemployment and Medicaid, COVID and Medicaid, COVID_capita and Medicaid, COVID_death and Medicaid and COVID_death capita and Medicaid}, j is a county in each state $s \in S$.


In [ ]:
#Normalize function to normalize the vulnerability values to be able to compare them

def normalize(dict_1):
    
    result = {}
    min_data = {s: min(dict_1[j] for j in location if (j,s) in cartesian_pro_county_state) for s in State }
    max_data = {s: max(dict_1[j] for j in location if (j,s) in cartesian_pro_county_state) for s in State }
    
    for (j,s) in cartesian_pro_county_state:
        
        if (max_data[s] - min_data[s]) != 0 :
    
            result[j] = (dict_1[j] - min_data[s])/(max_data[s] - min_data[s])
        
        else:
            result[j] = 1
    
    return result 

# Percentile Rank

<a href="https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.percentileofscore.html"> The function scipy.stats.percentileofscore (a, score, kind='rank')   </a>
computes the percentile rank of a score relative to a list of scores. 
"rank": Average percentage ranking of score. In case of multiple matches, average the percentage rankings of all matching scores.

In [ ]:
from scipy import stats

# Calculate percentile ranks

def percentile_ranks(data):
    x = {s: [] for s in State}

    for (j,s) in cartesian_pro_county_state:
         
        x[s].append(data[j])
    
    
    
    percentile_ranks = {i: stats.percentileofscore(x[s], data[i], 'rank') for (i,s) in cartesian_pro_county_state}

    return percentile_ranks

In [ ]:
# Write timestamp 

time_stamp = time.strftime('%m-%d-%Y %H:%M:%S')
with open('Output/time_stamp.csv','w') as f:
    w = csv.writer(f)
    now = time.strftime('%m/%d/%Y %H:%M:%S')
    w.writerow(['time',now])
    

In [ ]:
#print (Medicaid_demand)

In [ ]:
#Write a function to order the dicts
def order_k(dict_1):
    dict_2 = {}
    for m in location:
        if m in dict_1.keys():
            dict_2[m] = dict_1[m]
        else:
            dict_2[m] = 0
    
    return dict_2
            

In [ ]:


Medicaid_demand = order_k(Medicaid_demand)#{m: Medicaid_demand[m] for m in location}
COVID_14days = order_k(COVID_14days)#{m: COVID_14days[m] for m in location}
SVI_county = order_k(SVI_county)#{m: SVI_county[m] for m in location}
YPLL = order_k(YPLL)#{m: YPLL[m] for m in location}
Unemployment = order_k(Unemployment)#{m: Unemployment[m] for m in location}
Unemployment_capita = order_k(Unemployment_capita)
Covid_capita = order_k(Covid_capita)#{m: Unemployment_capita[m] for m in location}
Covid_death_capita = order_k(Covid_death_capita)

In [ ]:
# Write file allocation with each strategies for each county 

Strategies = ["Medicaid_demand", "Medicaid_capita", "Covid", "SVI"
              , "YPLL","Unemployment", "Unemployment_capita", "Covid_capita",  "Covid_death_capita" ]

fieldnames = []  
fieldnames.append('County_FIPS')


SVI_values = {i:SVI_county[i] for i in location}
s_count = 1
for s in Strategies:   
    fieldnames.append('Proportional_allocation_to_' + s)
    fieldnames.append(s)
    fieldnames.append('Percentile_ranks_' + s)

    
        

writefile = 'Output/County_level_proportional_allocation_for_all_policies.csv'
with open( writefile, 'w' ) as f:
    writer = csv.writer(f)                
    writer.writerow(fieldnames)
    for row in zip(location
                   , Proportional_to_medicaid.values(),         Medicaid_demand.values(),      percentile_ranks(Medicaid_demand).values()
                   , Proportional_to_medicaid_cap.values(),     Medicaid_capita.values(),      percentile_ranks(Medicaid_capita).values()
                   , Proportional_to_covid.values(),            COVID_14days.values(),         percentile_ranks(COVID_14days).values()
                   , Proportional_to_SVI.values(),              SVI_county.values(),           percentile_ranks(SVI_county).values()
                   , Proportional_to_YPLL.values(),             YPLL.values(),                 percentile_ranks(YPLL).values()
                   , Proportional_to_unemployment.values(),     Unemployment.values(),         percentile_ranks(Unemployment).values()
                   , Proportional_to_unemployment_cap.values(), Unemployment_capita.values(),  percentile_ranks(Unemployment_capita).values()
                   , Proportional_to_covid_capita.values(),     Covid_capita.values(),         percentile_ranks(Covid_capita).values()
                   , Proportional_to_covid_death_cap.values(),  Covid_death_capita.values(),   percentile_ranks(Covid_death_capita).values() ):                    
       
        writer.writerow(row)

In [ ]:
writefile = 'Output/State_level_allocation.csv'

cl = ['State', 'CHW_allocation']
with open( writefile, 'w' ) as f:
    writer = csv.writer(f)                
    writer.writerow(cl)
    for row in zip( State, Medicaid_demand_state.values()):
        writer.writerow(row)